# Testing de SQLite
Vamos a probar como gestiona python una base de sql como SQLite local

Python al parecer viene preinstalado con sqlite3 y tiene manejo de bases de datos locales por defecto.

In [1]:
import sqlite3

# Conectar a un archivo de base de datos (se crea si no existe). Genera un objeto conexion
conn = sqlite3.connect("mi_db.sqlite")

# Genera un cursor, es como una cmd de la base de datos
cur = conn.cursor()

# Se pueden ejecutar instrucciones SQL estandar con el cursor
cur.execute("""
CREATE TABLE IF NOT EXISTS catalogo (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre_producto TEXT NOT NULL,
    url_pccomp TEXT,
    url_pcbox TEXT,
    url_mediamarkt TEXT
)
""")

# Permite usar sustitucion por tuplas. Cambia todo "?" por (a,b,c,...) en orden de aparicion. SELECT ? (a) -> SELECT a
cur.execute("INSERT INTO catalogo (nombre_producto,url_pccomp,url_pcbox,url_mediamarkt) VALUES (?,?,?,?)",
    ("Tarjeta gráfica - MSI GeForce RTX 5060 VENTUS 2X OC, GDDR7, PCI Express 8GB",
    "https://www.mediamarkt.es/es/product/_tarjeta-grafica-msi-geforce-rtx-5060-ventus-2x-oc-gddr7-pci-express-x16-50-155816180.html",
    "https://www.pccomponentes.com/tarjeta-grafica-msi-geforce-rtx-5060-ventus-2x-oc-8gb-gddr7-reflex-2-rtx-ai-dlss4-blanca",
    "https://www.pcbox.com/912-v537-002-tarjeta-grafica-msi-nvidia-geforce-rtx-5060-gddr7-8gb-hdmi-dport/p"))

# Hay que confirmar la transaccion, no tiene autocommit por defecto
conn.commit()

# Si se ejecuta una consulta hay que sacar la respuesta de cursor.fetchall. Devuelve una lista[] de filas, iterar con un for simple
cur.execute("SELECT * FROM catalogo")
for fila in cur.fetchall():
    print(fila)

# Acordarse de cerrar la conexión, sino se puede colgar el motor interno con muchas conexiones abiertas
conn.close()

(1, 'Tarjeta gráfica - MSI GeForce RTX 5060 VENTUS 2X OC, GDDR7, PCI Express 8GB', 'https://www.mediamarkt.es/es/product/_tarjeta-grafica-msi-geforce-rtx-5060-ventus-2x-oc-gddr7-pci-express-x16-50-155816180.html', 'https://www.pccomponentes.com/tarjeta-grafica-msi-geforce-rtx-5060-ventus-2x-oc-8gb-gddr7-reflex-2-rtx-ai-dlss4-blanca', 'https://www.pcbox.com/912-v537-002-tarjeta-grafica-msi-nvidia-geforce-rtx-5060-gddr7-8gb-hdmi-dport/p')


Con esto ya de por si se puede hacer de todo, pero el output va a ser horrible con más filas.

Aunque sea matar moscas a cañonazos por lo potente que es, pandas integra consultas de sql y las deja bonitas:

In [2]:
import pandas as pd

# Volvemos a crear la Conexion con el mismo nombre
conn = sqlite3.connect("mi_db.sqlite")

# Creamos un dataframe de pandas, aparte de tener muchas herramientas de estadistica es que se imprime bonito
# Pandas usa su propia funcion para hacer busquedas en una Conexion
df = pd.read_sql_query("SELECT * FROM catalogo", conn)

# Se puede imprimir el objeto directmente pero asi se ve mejor
print(df.to_string())
conn.close()

   id                                                              nombre_producto                                                                                                                       url_pccomp                                                                                                               url_pcbox                                                                                         url_mediamarkt
0   1  Tarjeta gráfica - MSI GeForce RTX 5060 VENTUS 2X OC, GDDR7, PCI Express 8GB  https://www.mediamarkt.es/es/product/_tarjeta-grafica-msi-geforce-rtx-5060-ventus-2x-oc-gddr7-pci-express-x16-50-155816180.html  https://www.pccomponentes.com/tarjeta-grafica-msi-geforce-rtx-5060-ventus-2x-oc-8gb-gddr7-reflex-2-rtx-ai-dlss4-blanca  https://www.pcbox.com/912-v537-002-tarjeta-grafica-msi-nvidia-geforce-rtx-5060-gddr7-8gb-hdmi-dport/p


Lo único que pandas no sirve para operaciones CRUD (escritura y borrado), para eso hay que seguir con sqlite3

In [ ]:
from datetime import datetime # Si, es datetime.datetime yo tampoco se porque
import time

conn = sqlite3.connect("mi_db.sqlite")
cur = conn.cursor()

# Creamos una primary key compuesta
cur.execute("""
CREATE TABLE IF NOT EXISTS historial (
    id INTEGER NOT NULL,
    timestamp DATE NOT NULL,
    precio INTEGER,
    PRIMARY KEY (id, timestamp)
)
""")

# Vamos a crear nuestro timestamp, sello temporal del segundo donde se ejecuta algo
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f") # formatea el timestamp a algo legible

# Permite usar sustitucion por tuplas. Cambia todo "?" por (a,b,c,...) en orden de aparicion. SELECT ? (a) -> SELECT a
cur.execute("INSERT INTO historial (id,timestamp,precio) VALUES (?,?,?)", (1,timestamp,105.6))

 # Esperamos dos segundos para que haya diferencia
time.sleep(2.0)

timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")

cur.execute("INSERT INTO historial (id,timestamp,precio) VALUES (?,?,?)", (1,timestamp,107.1))

# Hay que confirmar la transaccion, no tiene autocommit por defecto
conn.commit()

cur.execute("SELECT * FROM historial")
for fila in cur.fetchall():
    print(fila)

# Vamos a borrar ese último, intentar usar algo mejor que precio para borrar
cur.execute("DELETE FROM historial WHERE precio = ?", (107.1,)) # Coma al final para indicar tupla de elemento unitario

conn.commit()

print("\n\n\n") # Hacemos hueco en la terminal

cur.execute("SELECT * FROM historial")
for fila in cur.fetchall():
    print(fila)

conn.close()

(1, '2025-10-03 17:58:49.174830', 105.6)
(1, '2025-10-03 17:58:51.175968', 107.1)




(1, '2025-10-03 17:58:49.174830', 105.6)


Para testing muy útil instalarse alguna extensión como SQLite Viewer de Florian Klampfer para tener una GUI de la base de datos

Por desgracia, SQL es necesario para que nuestro backend haga las instrucciones en python :(